In [501]:
import pandas as pd
import json
import pickle
import os
import itertools
from ast import literal_eval
import numpy as np
import sys
from datetime import datetime
from scipy import stats
from IPython.display import clear_output, display
import subprocess
from datetime import datetime
pd.options.display.float_format = '{:.0f}'.format

In [502]:
import pandas as pd
import numpy as np
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
french_stopwords = list(fr_stop)
from scipy import stats
import math
import string
import igraph as ig

In [503]:
sys.path.append("/home/serge/proj/streamer/")
from fun import *

print("")

In [504]:
#Functions

In [505]:
def BuildGraphFrom2DF(FinalInf,graphdf):

    """
    Input : un dataframe des nodes
    Input : un dataframe des links
    Output : un objet graph igraph
    """
    
    g = ig.Graph(directed=False)
    nodesids = FinalInf.GRAPHID.values
    g.add_vertices(nodesids)

    g.vs["AUTHORDESCRIPTION"] = FinalInf.AUTHORDESCRIPTION.tolist()
    g.vs["AUTHORFNAME"] = FinalInf.AUTHORFNAME.tolist()
    g.vs["AUTHORID"] = FinalInf.AUTHORID.tolist()
    g.vs["AUTHORNAME"] = FinalInf.AUTHORNAME.tolist()
    g.vs["AUTHORFOLLOWERS"] = FinalInf.AUTHORFOLLOWERS.tolist()

    edgeslist = graphdf[["GRAPHIDA","GRAPHIDB"]].values.tolist()
    g.add_edges(edgeslist)
    g.es["weight"] = graphdf.f.tolist()

    return g


def DeleteSomeNodes(g,AttributeName,ThresholdFollowersInf,ThresholdFollowersSup):

    """
    Input : un objet igraph graph
    Input : borne inferieur et superieur
    Input : Attribute Name
    Output : un objet igraph graph
    """
    
    
    to_delete_ids = [node for node in g.vs if node[AttributeName]<ThresholdFollowersInf]
    g.delete_vertices(to_delete_ids)

    to_delete_ids = [node for node in g.vs if node[AttributeName]>ThresholdFollowersSup]
    g.delete_vertices(to_delete_ids)

    return g

In [506]:
#Load and clean

In [507]:
graphdf = pd.read_csv("graph1581156942.csv")
graphdf.sort_values(by="f",ascending=False,inplace=True)
nodesids = np.unique(np.hstack((graphdf.a.values,graphdf.b.values)))

FinalInf = PickleLoad("FinalInf.pkl")
FinalInf = FinalInf[FinalInf.AUTHORID.isin(nodesids)]
FinalInf = FinalInf.reset_index(drop=True).reset_index()
FinalInf.rename(columns = {"index":"GRAPHID"},inplace=True)

graphdf = graphdf.merge(FinalInf[["GRAPHID","AUTHORID"]],how="left",left_on="a",right_on="AUTHORID").rename(columns={"GRAPHID":"GRAPHIDA"})
graphdf = graphdf.merge(FinalInf[["GRAPHID","AUTHORID"]],how="left",left_on="b",right_on="AUTHORID").rename(columns={"GRAPHID":"GRAPHIDB"})
graphdf.drop(columns=["AUTHORID_x","AUTHORID_y"],inplace = True)

In [508]:
#Graph

In [544]:
ThresholdFollowersInf = 20000
ThresholdFollowersSup = 80000
AttributeName = "AUTHORFOLLOWERS"
g = BuildGraphFrom2DF(FinalInf,graphdf)
g = DeleteSomeNodes(g,AttributeName,ThresholdFollowersInf,ThresholdFollowersSup)
todelete = [node for node in g.vs if node.degree()==0]
g.delete_vertices(todelete)
todelete = [edge for edge in g.es if edge["weight"]>4]
g.delete_edges(todelete)

In [545]:
clustering = g.community_multilevel(weights="weight")

# Récupérer les graph csv pour la vue globale

In [91]:
def ExtractNextBorne(SelectedID,x,verbose=False):

    try:
        InfValue = x[SelectedID][1]
        if verbose:
            print(InfValue)
        
        ChampDesPossibles = np.array([item[0] for item in x[SelectedID + 1 :len(x)]])
        if verbose:
            print(ChampDesPossibles)
        
        Distance = np.abs(InfValue - ChampDesPossibles)
        if verbose:
            print(Distance)
        
        idres = np.argmin(Distance) + SelectedID + 1 
        if verbose:
            print(idres)
        
        res = x[idres]
        if verbose:
            print(res)
        
    except:
        idres = None
        res = None
    
    return idres,res

In [93]:
def ExtractAllBornes(x):

    """
    Input : liste de tuples représentant les bornes
    Output : liste de tuples représentant les bornes les plus revelantes
    """
    
    x.sort(key = lambda a : a[1], reverse = False)
    SelectedID = 0
    borne = x[SelectedID]
    L = [borne]

    while True:
        idres,res = ExtractNextBorne(SelectedID,x)
        if idres is not None:
            SelectedID = idres
            L.append(res)
        else:
            break

    return L

In [94]:
x = [(0,10),
 (2,12),
 (4,14),
 (6,16),
 (8,18),
 (4,11),
 (12,22),
 (14,24),
 (16,26),
 (180,280),
 (200,300),
 (220,320)]


In [124]:
# Definition de la liste des bornes
BornesList = [(v["start"],v["tm"]) for k,v in gdic.items()]

# Extraction des bonnes bornes
MyBornes = ExtractAllBornes(x)

# Extraction de la TimeMarkList
TimeMarkList = [item[1] for item in MyBornes]

# Création du global links dataframe
L = []
ListOfSelectedDataFrames = [item["linksdf"] for item in gdic.keys() if item in TimeMarkList]
GlobalLinksDF = pd.concat(ListOfSelectedDataFrames,axis=0,sort=True)
GlobalStatsDF = GlobalLinksDF.groupby(["a","b"])["f"].sum().reset_index()

# Création de FinalInf, GlobalStatsDF
# ...

# Parameters du filtre du graph
g = BuildGraphFrom2DF(FinalInf,graphdf)

# Filtrage du graph g
# ...

# Clustering du graph
clustering = g.community_multilevel(weights="weight")

**Bleckwenn**, AntiFraud Company

* Goal : improve global graph algorithms performance
 * Developpement : subgraph extraction model
 * Example : get "relevant" ego graph of a node, bypassing hubs and dead "links" and outliers "links"


* Goal : improve fraudulent operations detection model
 * Developpement : build bunch of features that would characterize a transaction
 * Example : age of an account, past operations, countries involved, banks involved, validation


* Goal : detect events by listenning meta data communications within a group of people 
 * Developpement : events extraction model via twitter data
 * Example : giving the activity on twitter this monday, something unusual happens within this community
 * Justification : crime is an activity played in a group. One fraudulent transaction implies a community of criminals. 

**Suneris**, Military Intelligence

* Goal : extract spatiotemporal habits
 * Developpement : Hierarchical spatial areas clustering
 * Developpement : Hierarchical temporal patterns cycles detection
 * Example : Where and when can I find him?


* Goal : Guessing phone numbers
 * Developpement : Matching habits model
 * Example : If he got an other number, what it could be?


* Goal : track identified spatiotemporal habits in order to raise alert when something unusual happens
 * Developpement : Temporal patterns breakouts detection model
 * Exemple : he used to go there with a high confidence, but today no, something might happen
 

**Bouygues**,  Telecom

* Goal : detect types of failures of the newtork by analysing sequence of the hotline suggested actions
 * Developpement : extract frequent subsequences patterns
 * Developpement : model that predict type of failures from patterns
 * Developpement : application that run the predictive model over a sliding window and may raise alert in case of dominating type of failures

**Orange**, Telecom
 * Goal : enrich a model that explains the customer dissatisfaction
  * Developpement : features extraction based on sequence of events that users trigger via their remote control
  * Example : the subsequence [*TurnOn - TurnOff - TurnOn*] in short period of time is highly correlated to dissatisfaction, so we want to incorporate this feature into the model